In [10]:
import os
import pandas as pd
import numpy as np

In [11]:
rawdata_loc = "D:\Projects_IndranilDa\Customer_Acq_Model\customer_acq_model\data\\01_raw"
reporting_loc = "D:\Projects_IndranilDa\Customer_Acq_Model\customer_acq_model\data\\03_reporting" 
intermediate_loc = "D:\Projects_IndranilDa\Customer_Acq_Model\customer_acq_model\data\\02_intermediate"

In [13]:
## pulling Feature Engineered Data
data = pd.read_csv(f"{intermediate_loc}\\feature_engineered_data.csv")
data.head(5)

,store_sales(in millions),store_cost(in millions),unit_sales(in millions),marital_status,gender,total_children,education,member_card,occupation,houseowner,...,"media_type_Daily Paper, Radio","media_type_Daily Paper, Radio, TV",media_type_In-Store Coupon,media_type_Product Attachment,media_type_Radio,media_type_Street Handout,media_type_Sunday Paper,"media_type_Sunday Paper, Radio","media_type_Sunday Paper, Radio, TV",media_type_TV
0,0.305667,0.267690,0.6,1,1,0.2,1.00,0.666667,0.75,1,...,1,0,0,0,0,0,0,0,0,0
1,0.223561,0.254222,0.4,1,0,0.0,0.50,0.000000,1.00,1,...,1,0,0,0,0,0,0,0,0,0
2,0.141455,0.125312,0.2,0,1,0.8,1.00,0.666667,0.50,0,...,1,0,0,0,0,0,0,0,0,0
3,0.141455,0.106072,0.2,1,1,0.4,0.75,1.000000,0.50,1,...,0,0,1,0,0,0,0,0,0,0
4,0.159304,0.132256,0.4,1,0,0.0,1.00,1.000000,0.75,0,...,0,0,0,0,1,0,0,0,0,0
